In [1]:
import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings

# from agentics import agentics as AG

from agentics.core.agentics import AG

from agentic_energy.schemas import BatteryParams, SolveRequest,DayInputs


import numpy as np
# import matplotlib.pyplot as plt
# from agentic_energy.milp.milp_mcp_server import records_to_arrays, solve_daily_milp
from agentic_energy.data_loader import EnergyDataLoader
from agentic_energy.schemas import BatteryParams, DayInputs, EnergyDataRecord, SolveFromRecordsRequest

Forecast Engine using device: cpu


In [2]:
import pandas as pd
italy_data = pd.read_csv("agentic_energy/data/ITALY_data_with_decisions.csv")

In [3]:
#remove columns soc and soc_previous if they exist
if 'soc' in italy_data.columns:
    italy_data = italy_data.drop(columns=['soc'])
if 'soc_prev' in italy_data.columns:
    italy_data = italy_data.drop(columns=['soc_prev'])

italy_data.head()


,timestamps,prices,consumption,fossil_generation,wind_generation,month,decisions
0,2022-01-01 00:00:00,170.28,22.40,11.05,6.81,winter,0
1,2022-01-01 01:00:00,155.72,20.79,10.36,6.36,winter,-1
2,2022-01-01 02:00:00,147.09,19.85,9.60,6.12,winter,-1
3,2022-01-01 03:00:00,91.00,19.12,9.16,5.94,winter,0
4,2022-01-01 04:00:00,104.00,19.08,8.43,6.02,winter,1


In [4]:
# # randomly put NaN in decisions column to simulate missing decisions for around 95% of the data
# np.random.seed(42)
# mask = np.random.rand(len(italy_data)) < 0.95
# italy_data.loc[mask, 'decisions'] = np.nan

# just have the first 60 rows with decision data and replace the rest with NaN
italy_data.loc[1*24:, 'decisions'] = np.nan

In [5]:
italy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17496 entries, 0 to 17495
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timestamps         17496 non-null  object 
 1   prices             17496 non-null  float64
 2   consumption        17496 non-null  float64
 3   fossil_generation  17496 non-null  float64
 4   wind_generation    17496 non-null  float64
 5   month              17496 non-null  object 
 6   decisions          24 non-null     float64
dtypes: float64(5), object(2)
memory usage: 956.9+ KB


In [6]:
italy_data

,timestamps,prices,consumption,fossil_generation,wind_generation,month,decisions
0,2022-01-01 00:00:00,170.28,22.40,11.05,6.81,winter,0.0
1,2022-01-01 01:00:00,155.72,20.79,10.36,6.36,winter,-1.0
2,2022-01-01 02:00:00,147.09,19.85,9.60,6.12,winter,-1.0
3,2022-01-01 03:00:00,91.00,19.12,9.16,5.94,winter,0.0
4,2022-01-01 04:00:00,104.00,19.08,8.43,6.02,winter,1.0
...,...,...,...,...,...,...,...
17491,2023-12-30 23:00:00,108.62,22.06,14.66,6.12,winter,NaN
17492,2023-12-31 00:00:00,109.12,20.71,13.25,5.19,winter,NaN
17493,2023-12-31 01:00:00,107.06,19.26,12.08,4.58,winter,NaN
17494,2023-12-31 02:00:00,105.45,18.41,10.90,4.26,winter,NaN


In [7]:
capacity = 1.0  # MWh
italy_data["battery_capacity_mwh"] = capacity
italy_data["battery_chargingrate_mw"] = capacity/4
italy_data["battery_dischargingrate_mw"] = capacity/4
italy_data["battery_soc_init_just_the_first_timestamp_of_the_whole_dataset"] = 0.50
italy_data["battery_state_of_charge_soc_min"] = 0.0
italy_data["battery_state_of_charge_soc_max"] = 1.0
italy_data["battery_charging_efficiency"] = 0.95
italy_data["battery_discharging_efficiency"] = 0.95
italy_data["battery_soc_try_to_maintain_target_end_of_each_day"] = 0.50
italy_data.head()

,timestamps,prices,consumption,fossil_generation,wind_generation,month,decisions,battery_capacity_mwh,battery_chargingrate_mw,battery_dischargingrate_mw,battery_soc_init_just_the_first_timestamp_of_the_whole_dataset,battery_state_of_charge_soc_min,battery_state_of_charge_soc_max,battery_charging_efficiency,battery_discharging_efficiency,battery_soc_try_to_maintain_target_end_of_each_day
0,2022-01-01 00:00:00,170.28,22.40,11.05,6.81,winter,0.0,1.0,0.25,0.25,0.5,0.0,1.0,0.95,0.95,0.5
1,2022-01-01 01:00:00,155.72,20.79,10.36,6.36,winter,-1.0,1.0,0.25,0.25,0.5,0.0,1.0,0.95,0.95,0.5
2,2022-01-01 02:00:00,147.09,19.85,9.60,6.12,winter,-1.0,1.0,0.25,0.25,0.5,0.0,1.0,0.95,0.95,0.5
3,2022-01-01 03:00:00,91.00,19.12,9.16,5.94,winter,0.0,1.0,0.25,0.25,0.5,0.0,1.0,0.95,0.95,0.5
4,2022-01-01 04:00:00,104.00,19.08,8.43,6.02,winter,1.0,1.0,0.25,0.25,0.5,0.0,1.0,0.95,0.95,0.5


In [8]:
italy_data=italy_data.iloc[:24*5]
italy_data.shape

(120, 16)

In [9]:
italy_data.to_csv("agentic_energy/data/ITALY_data_fewshots.csv", index=False)

In [10]:
italy_dat = AG.from_csv("agentic_energy/data/ITALY_data_fewshots.csv")

In [11]:
italy_dat.states[:5]

[AType#timestamps:prices:consumption:fossil_generation:wind_generation:month:decisions:battery_capacity_mwh:battery_chargingrate_mw:battery_dischargingrate_mw:battery_soc_init_just_the_first_timestamp_of_the_whole_dataset:battery_state_of_charge_soc_min:battery_state_of_charge_soc_max:battery_charging_efficiency:battery_discharging_efficiency:battery_soc_try_to_maintain_target_end_of_each_dayOptional(timestamps='2022-01-01 00:00:00', prices='170.28', consumption='22.4', fossil_generation='11.05', wind_generation='6.81', month='winter', decisions='0.0', battery_capacity_mwh='1.0', battery_chargingrate_mw='0.25', battery_dischargingrate_mw='0.25', battery_soc_init_just_the_first_timestamp_of_the_whole_dataset='0.5', battery_state_of_charge_soc_min='0.0', battery_state_of_charge_soc_max='1.0', battery_charging_efficiency='0.95', battery_discharging_efficiency='0.95', battery_soc_try_to_maintain_target_end_of_each_day='0.5'),
 AType#timestamps:prices:consumption:fossil_generation:wind_gene

In [31]:
italy_dat.states[0]

AType#timestamps:prices:consumption:fossil_generation:wind_generation:month:decisions:battery_capacity_mwh:battery_chargingrate_mw:battery_dischargingrate_mw:battery_soc_init_just_the_first_timestamp_of_the_whole_dataset:battery_state_of_charge_soc_min:battery_state_of_charge_soc_max:battery_charging_efficiency:battery_discharging_efficiency:battery_soc_try_to_maintain_target_end_of_each_dayOptional(timestamps='2022-01-01 00:00:00', prices='170.28', consumption='22.4', fossil_generation='11.05', wind_generation='6.81', month='winter', decisions='0.0', battery_capacity_mwh='1.0', battery_chargingrate_mw='0.25', battery_dischargingrate_mw='0.25', battery_soc_init_just_the_first_timestamp_of_the_whole_dataset='0.5', battery_state_of_charge_soc_min='0.0', battery_state_of_charge_soc_max='1.0', battery_charging_efficiency='0.95', battery_discharging_efficiency='0.95', battery_soc_try_to_maintain_target_end_of_each_day='0.5')

In [32]:
battery_details = BatteryParams(
    capacity_MWh=20.0, soc_init=0.5, soc_min=0.10, soc_max=0.90,
    cmax_MW=6.0, dmax_MW=6.0, eta_c=0.95, eta_d=0.95, soc_target=0.5
)

days = DayInputs(
    prices_buy=np.array([float(rec.prices) for rec in italy_dat.states]),
    prices_sell=np.array([float(rec.prices) for rec in italy_dat.states]),
    allow_export=True,  # allow export every hour
    demand_kw=np.array([float(rec.consumption) for rec in italy_dat.states]),
    dt_hours=1.0,
    decisions=np.array([float(rec.decisions) for rec in italy_dat.states if (rec.decisions) is not ''])  # fill NaN with 0.0 (idle)
)

solve = SolveRequest(
    battery=battery_details,
    day=days,
    solver=None,
    solver_opts=None
    # solver_opts = {
    #     "TimeLimit": 300,        # Maximum solve time in seconds
    #     "MIPGap": 0.01,         # Stop when gap between best solution and bound < 1%
    #     "Threads": 4,           # Number of threads to use
    #     "OutputFlag": 1,        # 1 = show solver output, 0 = silent
    #     "LogToConsole": 1       # Print log to console
    # }
)

source = AG(atype=SolveRequest, states=[solve])

target = await(italy_dat("decisions") << source)
# Agentics does not support the above yet.

IndexError: list index out of range

In [34]:
italy_dat("decisions")

AG(atype=<class 'agentics.core.agentics.decisions'>, crew_prompt_params={'role': 'Task Executor', 'goal': 'You execute tasks', 'backstory': 'You are always faithful and provide only fact based answers.', 'expected_output': 'Described by Pydantic Type'}, instructions='Generate an object of the specified type from the following input.', llm=<crewai.llm.LLM object at 0x000001BAE0237BD0>, max_iter=3, prompt_template=None, reasoning=None, skip_intentional_definition=False, states=[decisions(decisions='0.0'), decisions(decisions='-1.0'), decisions(decisions='-1.0'), decisions(decisions='0.0'), decisions(decisions='1.0'), decisions(decisions='1.0'), decisions(decisions='0.0'), decisions(decisions='-1.0'), decisions(decisions='1.0'), decisions(decisions='1.0'), decisions(decisions='1.0'), decisions(decisions='0.0'), decisions(decisions='0.0'), decisions(decisions='-1.0'), decisions(decisions='0.0'), decisions(decisions='1.0'), decisions(decisions='0.0'), decisions(decisions='0.0'), decisions(d

In [24]:
float(italy_dat.states[0].prices)

170.28

In [13]:
str(type(italy_dat.states[0])).split('.')[-1].split(':')

['AType#timestamps',
 'prices',
 'consumption',
 'fossil_generation',
 'wind_generation',
 'month',
 'decisions',
 'battery_capacity_mwh',
 'battery_chargingrate_mw',
 'battery_dischargingrate_mw',
 'battery_soc_init_just_the_first_timestamp_of_the_whole_dataset',
 'battery_state_of_charge_soc_min',
 'battery_state_of_charge_soc_max',
 'battery_charging_efficiency',
 'battery_discharging_efficiency',
 "battery_soc_try_to_maintain_target_end_of_each_dayOptional'>"]

In [14]:
columns= ['timestamps',
 'prices',
 'consumption',
 'fossil_generation',
 'wind_generation',
 'month',
 'battery_capacity_mwh',
 'battery_chargingrate_mw',
 'battery_dischargingrate_mw',
 'battery_soc_init_just_the_first_timestamp_of_the_whole_dataset',
 'battery_state_of_charge_soc_min',
 'battery_state_of_charge_soc_max',
 'battery_charging_efficiency',
 'battery_discharging_efficiency',
 "battery_soc_try_to_maintain_target_end_of_each_day"]

In [17]:
italy_data_with_decisions = await (italy_dat("decisions") << italy_dat('timestamps',
 'prices',
 'consumption',
 'fossil_generation',
 'wind_generation',
 'month',
 'battery_capacity_mwh',
 'battery_chargingrate_mw',
 'battery_dischargingrate_mw',
 'battery_soc_init_just_the_first_timestamp_of_the_whole_dataset',
 'battery_state_of_charge_soc_min',
 'battery_state_of_charge_soc_max',
 'battery_charging_efficiency',
 'battery_discharging_efficiency',
 "battery_soc_try_to_maintain_target_end_of_each_day"))
print(italy_data_with_decisions.pretty_print())

Output()

aType : <class 'agentics.core.agentics.decisions'>
decisions: '0.0'

decisions: '-1.0'

decisions: '-1.0'

decisions: '0.0'

decisions: '1.0'

decisions: '1.0'

decisions: '0.0'

decisions: '-1.0'

decisions: '1.0'

decisions: '1.0'

decisions: '1.0'

decisions: '0.0'

decisions: '0.0'

decisions: '-1.0'

decisions: '0.0'

decisions: '1.0'

decisions: '0.0'

decisions: '0.0'

decisions: '-1.0'

decisions: '-1.0'

decisions: '-1.0'

decisions: '-1.0'

decisions: '0.0'

decisions: '1.0'

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''

decisions: ''


In [18]:
italy_data_with_decisions.states

[decisions(decisions='0.0'),
 decisions(decisions='-1.0'),
 decisions(decisions='-1.0'),
 decisions(decisions='0.0'),
 decisions(decisions='1.0'),
 decisions(decisions='1.0'),
 decisions(decisions='0.0'),
 decisions(decisions='-1.0'),
 decisions(decisions='1.0'),
 decisions(decisions='1.0'),
 decisions(decisions='1.0'),
 decisions(decisions='0.0'),
 decisions(decisions='0.0'),
 decisions(decisions='-1.0'),
 decisions(decisions='0.0'),
 decisions(decisions='1.0'),
 decisions(decisions='0.0'),
 decisions(decisions='0.0'),
 decisions(decisions='-1.0'),
 decisions(decisions='-1.0'),
 decisions(decisions='-1.0'),
 decisions(decisions='-1.0'),
 decisions(decisions='0.0'),
 decisions(decisions='1.0'),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions(decisions=''),
 decisions

In [15]:
italy_data_with_decisions = await italy_dat.self_transduction(
    columns, ["decisions"],
    instructions=
    f'''Fill in the missing battery decisions for energy arbitrage based on the provided data. 
    Use the existing decisions as examples to infer the missing values. Ensure that the decisions respect the battery constraints such as capacity, 
    charging/discharging rates, and state of charge limits. The goal is to optimize the battery usage for cost savings while adhering to these constraints.
    
    You are solving a daily battery scheduling optimization problem using Mixed Integer Linear Programming (MILP). 
                You are given a request object containing:
                - Hourly energy prices for buying and selling electricity.
                - Hourly electricity demand from a building or system.
                - Battery technical parameters including capacity_MWh: {capacity}, charge/discharge power limits cmax_MW, dmax_MW, efficiencies - eta_c, eta_d, and state-of-charge soc_max, soc_min bounds.

                Your task is to:
                1. Determine the hourly charge , discharge, grid import, grid export, and SoC schedule for 24 hours.
                2. Minimize the total operational cost:
                    total_cost = Σ_t [ (price_buy[t] * import_MW[t] - price_sell[t] * export_MW[t]) * dt_hours ]
                3. Ensure all constraints are satisfied:
                - SoC at time t = SoC at time t-1 + (eta_c * charge_MW[t]  - discharge_MW[t] / eta_d ) * dt_hours  / capacity_MWh
                - soc_min ≤ SoC_t ≤ soc_max for all t
                - 0 ≤ charge_MW[t]  ≤ cmax_MW
                - 0 ≤ discharge_MW[t] ≤ dmax_MW 
                - import_MW[t] = max(0, demand_MW[t] + charge_MW[t]  - discharge_MW[t] - export_MW[t])
                - export_MW[t] ≥ 0 only if allow_export  = True
                - initialize the soc variable at soc_init at t=0, where t is the first hour of the day.
                - The battery SoC at the end of the day should reach soc_target (if provided), else soc_init.
                - Assume the battery can either charge  or discharge or stay idle in a given hour, not both. So try to schedule the battery in such a way.

                4. Output a JSON-compatible SolveResponse object with:
                - status: "success" or "failure"
                - message: optional diagnostic
                - objective_cost: the minimized total cost
                - charge_MW: list of hourly charge values (MW)
                - discharge_MW: list of hourly discharge values (MW)
                - import_MW: list of hourly grid import values (MW)
                - export_MW: list of hourly grid export values (MW)
                - soc: list of hourly state of charge values (fraction of capacity between 0 and 1)

                Make sure the final schedule satisfies all physical constraints and the objective function is minimized.
            '''
)
print(italy_data_with_decisions.pretty_print())

Output()

aType : <class 'agentics.core.atype.AType#timestamps:prices:consumption:fossil_generation:wind_generation:month:decisions:battery_capacity_mwh:battery_chargingrate_mw:battery_dischargingrate_mw:battery_soc_init_just_the_first_timestamp_of_the_whole_dataset:battery_state_of_charge_soc_min:battery_state_of_charge_soc_max:battery_charging_efficiency:battery_discharging_efficiency:battery_soc_try_to_maintain_target_end_of_each_dayOptional'>
timestamps: '2022-01-01 00:00:00'
prices: '170.28'
consumption: '22.4'
fossil_generation: '11.05'
wind_generation: '6.81'
month: winter
decisions: '0.0'
battery_capacity_mwh: '1.0'
battery_chargingrate_mw: '0.25'
battery_dischargingrate_mw: '0.25'
battery_soc_init_just_the_first_timestamp_of_the_whole_dataset: '0.5'
battery_state_of_charge_soc_min: '0.0'
battery_state_of_charge_soc_max: '1.0'
battery_charging_efficiency: '0.95'
battery_discharging_efficiency: '0.95'
battery_soc_try_to_maintain_target_end_of_each_day: '0.5'

timestamps: '2022-01-01 01:00

In [16]:
italy_data_with_decisions.states[0].decisions

'0.0'